In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 8, 6
import numpy as np 
import pickle
import glob 
import os 
import time , sys
from scipy import integrate
from scipy import interpolate
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator
from cycler import cycler
from mpl_toolkits.mplot3d import Axes3D
import vegas
#look at fixing later 
import warnings
warnings.filterwarnings('ignore')

from functions2 import * 

data_dir='/home/ben/Documents/Bonn-uni/internship/takeaway_data/' #'/users/bhastings/internship/data/MW/' # #
metallicity='mw'
file_ext= '.dat2'

import scipy
print scipy.__version__

0.18.1


In [2]:
# sort file names so that they are listed by mass, then rotation velocity 
m_val=lambda x : int(x.split('/')[-1].split('.')[0].split('-')[0][1:])
v_val= lambda x : int(x.split('/')[-1].split('.')[0].split('-')[1])

s1=sorted(glob.glob(data_dir +'*'+ file_ext), key=v_val)
data_files=sorted(s1, key= m_val)


#cut stars heavier than 40M, at they do not contribute to Be
data_files=filter(lambda x: m_val(x) < 30, data_files)
#print data_files

mass_list=[]
for f in data_files:
    mass_list=np.append(mass_list, int(m_val(f)))
    
mass_list=np.unique(mass_list)


print 'masses of models :\n',mass_list

masses of models :
[ 3.  5.  7.  9. 10. 12. 15. 20. 25.]


In [3]:
#load in velocity dstribution 
v_dist=np.poly1d([ 4.96085349e-13, -4.27023037e-10,  6.65435944e-08,  6.46442910e-06
  ,1.93953157e-03])
v_max_val=430

#set Be threshold 
Be_value=0.7

In [4]:
#set up IMF
#check that total CDF between mass limits is 1. If not eta0 value needs to be tweaked! 
global alpha, eta0
alpha=2.3
lower= 3
upper =25
eta0=1 / integrate_imf(lower,upper, alpha,1)


print "integral of IMF between ", lower,'and', upper, ',eta0 = ', eta0, ":"
print integrate_imf(lower , upper , alpha , eta0)

integral of IMF between  3 and 25 ,eta0 =  5.79034154251 :
1.0


# load data from models #

In [5]:
data_df=pd.DataFrame()
i=0
for fil in data_files:
    message="working on file " +str(i) + " / "+ str(len(data_files)) #+ " :time loop "+str(i2) + " / "+ str(n_vals)
    sys.stdout.write ('\r'+ message)
    i=i+1
    df_ms, T_ms= MS_finder2(fil)
    

        # index of time closest to t 
        #T_indx=(df_ms['1:t[s]']-t).abs().argmin()
        #t_i=df_ms['1:t[s]'].loc[T_indx]
        #V= df_ms['9:v_surf[km/s]'].loc[T_indx] 
        #if star is rotating 
    if v_val(fil) != 0:
        V_over_Vcrit=df_ms['9:v_surf[km/s]']/ df_ms['8:v_crit[km/s]']
        #print V_over_Vcrit
        V_initial=df_ms['9:v_surf[km/s]'][0:100].mean()
    else:
        V_over_Vcrit=0
        V_initial=0
        V=0

    ti_over_tms=df_ms['1:t[s]']/T_ms

    #df_ms=df_ms.append(pd.DataFrame([[f_name,m,V_initial,V, T_ms,V_over_Vcrit, ti_over_tms]],
                                                      #columns=['f-name','M','V_initial', 'V', 'T_MS', 'V/Vcrit', 't_i/T_MS']))

    df_ms['V/Vcrit']=V_over_Vcrit
    df_ms['t/Tms']=ti_over_tms
    df_ms['V_init']=V_initial
    df_ms['M']= m_val(fil)
    df_ms['V']=v_val(fil)
    df_ms['T_MS']= T_ms
    
    data_df=data_df.append(df_ms)

working on file 111 / 112

# find interpolation between T_MS, V_init and M #

In [6]:
df=data_df[data_df['1:t[s]']== 0]


#print df
x=np.array(df['V_init'])#[-20:]
y=np.array(df['M'])#[-20:]
z=np.array(df['T_MS'])#[-20:]

        #print x,y,z
f = interpolate.interp2d(x, y, z, kind='linear')

In [8]:
def V_min_Calc(m_i):
    b=np.array(map(lambda x: [x,t/f(x,m_i)[0]] , np.linspace(0,v_max_val,1e3)))

    V_min= b[(np.abs(b[:,1]-1)).argmin()]
    if V_min[1] > 0.99 and V_min[1] <1.01:
        V_min=V_min[0]
            
    elif V_min[1] >1.01:
        V_min=v_max_val
    elif V_min[1] <0.99:
        V_min=0
    return V_min

def V_critmin_Calc(m_i):

    #find masses in grid between m_i 
    m1= data_df[data_df['M'] <=  m_i].M.max()
    m2= data_df[data_df['M'] >  m_i].M.min()
    d11=data_df[(data_df['M'] == m1) ]
    d22=data_df[(data_df['M'] == m2) ]

    db=d11.append(d22)

    

    V_min= V_min_Calc(m_i)
    
    T=f(V_min,m_i)[0]
    
    if t/T >= 0.99: 
        return v_max_val
    
     #print m1, m2
    #for vi in np.unique(d22['V_init'])[1:]:
    #    dx=d22[d22['V_init']==vi]
    #    dx=dx[dx['V/Vcrit'] >= 0.05]
        #print dx['1:t[s]'].max()
    #    x=dx['1:t[s]']/dx['1:t[s]'].max()
    #    y=dx['V/Vcrit']
        #print x
        #plt.plot(dx['1:t[s]']/dx['1:t[s]'].max(), dx['V/Vcrit'], 'k-')
        
    #plt.title(m2)
    #plt.axvline(t/T)
    #plt.show()

    dm=pd.DataFrame()
    
    for vi in np.unique(db['V_init'])[1:]:
        di= db[(db['V_init']==vi)   ]
        #di=di[di['V/Vcrit'] >= 0.05]
        #di['t/tms']= (di['1:t[s]']/(di['1:t[s]'].max()))
        ti=((di['t/Tms']-(t/T)).abs()).argmin()
        #print 't/T' ,(t/T), di['1:t[s]']/(di['1:t[s]'].max())
        #print len((di['t/tms']-(t/T)).abs()), vi, len(di), t/T, ti
        #print  di[di['t/Tms'] == 0.0]
        dn= di.iloc[ti]#di[di['t/Tms'] == ti]
        dm=dm.append(dn)
        #plt.plot( dm['V_init'],dm['V/Vcrit'], 'ro')
    
    #plt.show()
    #df=dm[dm['t/Tms'] <= 1]
    df=dm

    #print df
    z=np.array(df['V_init'])#[-20:]
    x=np.array(df['M'])#[-20:]
    y=np.array(df['V/Vcrit'])#[-20:]

    #f_n = interpolate.interp2d(x, y, z, kind='linear')
    #m_list=np.linspace(m1,m2,500)
    #for i in m_list:
    #    plt.plot(i, (f_n(i, 0.7)), 'k.' )

    #plt.show()
    #print f_n(5, 0.7)
    
    straight_line1=interpolate.interp1d( df['V/Vcrit'][df.M ==m1],df['V_init'][df.M ==m1],
                                        fill_value="extrapolate", assume_sorted=True)
    straight_line2=interpolate.interp1d( df['V/Vcrit'][df.M ==m2],df['V_init'][df.M ==m2],
                                        fill_value="extrapolate", assume_sorted=True)

    plot= False#True 
    if plot==True :
    
        plt.plot( df['V_init'][df.M ==m1],df['V/Vcrit'][df.M ==m1], 'ro')
        plt.plot( df['V_init'][df.M ==m2],df['V/Vcrit'][df.M ==m2], 'go')
        plt.plot(straight_line1(np.linspace(0,1,20)), np.linspace(0,1,20), 'k-')
        plt.plot(straight_line2(np.linspace(0,1,20)), np.linspace(0,1,20), 'k-')
        plt.axhline(Be_value)
        plt.title(m_i)
        plt.show()
    
   
        for m in [m1,m2]:
            dx=data_df[data_df.M ==m]
            for vi in np.unique(dx['V_init'])[1:]:
                dl=dx[(dx['V_init']==vi)]
                #dx=dx[dx['V/Vcrit'] >= 0.05]
                #print dx['1:t[s]'].max()
                x=dl['t/Tms']
                y=dl['V/Vcrit']
                #print x
                plt.plot(x,y)
            plt.title(m)    
            plt.axvline(t/T)
            plt.show()
    
    #print m_i, t/T, t/1e6
    final_interp=interpolate.interp1d([m1,m2], 
                                [straight_line1(Be_value), straight_line2(Be_value)], 
                                fill_value="extrapolate") #, T/1e6, V_min
    
    return final_interp(m_i)

def M_MAX_calc2(data_df, t_val, v_max_value):
    dm=pd.DataFrame()
    d22t=pd.DataFrame()
    for mi in np.unique(data_df['M']):
        d=data_df[data_df['M']==mi]
        for vi in np.unique(d['V']):
            di= d[d['V']==vi]
            #ti=(di['t/Tms']-(1)).abs().argmin()
            #print ti, vi, len(di)
            #print di.iloc[-1]
            #print  di[di['t/Tms'] == 0.0]
            dn= di.iloc[-1]#di[di['t/Tms'] == ti]
            dm=dm.append(dn)
    #if ms lifetime of largest mass in dataframe rotating at highest velocity is less than time, give the largest mass
    
    #print f(v_max_value, dm['M'].max())
    if f(v_max_value, dm['M'].max()) >=t_val:
        return dm['M'].max()

    df=dm

    grid_x, grid_y = np.mgrid[0:t_val:1000j, 0: v_max_value:1000j]
    values = np.array(df['M'].astype(float))
    points=np.array(df[['1:t[s]','V_init' ]])


    grid_z0 = griddata(points, values, (grid_x, grid_y), method='linear')

    #print grid_y[0][950]
    #print grid_x[900][0]

    #look for values in range t/Tms >0.95 and v > 0.5 v_max 
    M_min_candidates=[]
    for x in range(995,1000-1):
        for y in range(995,1000-1):
            M_min_candidates=np.append(M_min_candidates,grid_z0[x][y])


    return M_min_candidates.max()
        #plot interpolation
    #plt.imshow(grid_z0, extent=(0.9,1,0.9,1), origin='lower')
    #plt.colorbar()
    #plt.title(str(t_val/1e6) +'Myr')
    #plt.xlabel('t/Tms')
    #plt.ylabel('V/max')
    #plt.show()
    
t=10e6
#M_max=M_MAX_calc2(data_df, t, 430)
#print 'M_MAX',M_max

start_time=time.time()
Q=1e3
for m_i in np.linspace(3,M_max,Q):
    V_critmin_Calc(m_i)
    #print m_i, V_critmin_Calc(m_i), V_min_Calc(m_i)
    
print '\ntime taken (mins)= ',   (time.time()- start_time)/60, 'for', Q, 'iterations'


time taken (mins)=  1.96439773242 for 1000.0 iterations


# define integrals #

In [16]:
def integrand_n(x):
        m_i=x[0]
        V_i=x[1]
        
        
        Vc_min= V_critmin_Calc(m_i)
        #print Vc_min
        
        i=integrate_vdist(Vc_min ,v_max_val, v_dist,v_max_val)/integrate_vdist(0 ,v_max_val, v_dist,v_max_val)
        
        i1=eta0 * m_i**-alpha
        #print 'Vc_min=', Vc_min, 'm=',np.round(m_i, 2), 't=', np.round(t/1e6,2), 'TMS=', np.round(T/1e6, 2)
        return i*i1
        
def integrand_N(x):
        #print t
        m_i=x[0]
        V_i=x[1]
        
        V_min= V_min_Calc(m_i)
        
        i=integrate_vdist(V_min ,v_max_val, v_dist,v_max_val)/integrate_vdist(0 ,v_max_val, v_dist,v_max_val)
        
        i1=eta0 * m_i**-alpha
        #print 'integrands',i, i1
        #print 'V_min=', V_min,'i',i, 'm=',np.round(m_i, 2), 't/TMS=',np.round(t/T,4), 't=', np.round(t/1e6,2), 'TMS=', np.round(T/1e6, 2)
        return i*i1
    

# compute integrals # 

In [ ]:
print 'STARTING'
df_N=pd.DataFrame()
start_time=time.time()
for t in (0,2,5,8,10,15,20,30,40,50,60,70,80,100,120,150, 200, 250,300,400,500):
    t=t*1e6
    M_max=M_MAX_calc2(t, v_max_val, 3,25 )
    integ = vegas.Integrator([[3,M_max], [0, 1]])
    result_N = integ(integrand_N, nitn=10, neval=1e3)
    print t/1e6,np.round(M_max,3), np.round(result_N.val,5)
    df_N=df_N.append(pd.DataFrame([[t,result_N.val]], columns=['t', 'f']))
print '\ntime taken (mins)= ',   (time.time()- start_time)/60

In [ ]:
print 'STARTING'
df_n=pd.DataFrame()

start_time=time.time()
for t in (0,2,5,8,10,15,20,30,40,50,60,70,80,100,120,150, 200, 250,300,400,500):
    t=t*1e6
    M_max=M_MAX_calc2(data_df, t, v_max_val)
    integ = vegas.Integrator([[3,M_max], [0, 1]])
    result_n = integ(integrand_n, nitn=10 ,neval=1e3)
    print t/1e6,np.round(M_max,3), np.round(result_n.val,5)
    df_n=df_n.append(pd.DataFrame([[t,result_n.val, Be_value]], columns=['t', 'f', 'Be_value']))
    
print '\ntime taken (mins)= ',   (time.time()- start_time)/60

STARTING


In [ ]:
df_n.to_pickle('./n_res.pk')
#df_N.to_pickle('./N_res.pk')
